In [3]:
import os
import re
import copy
import pickle

import urllib.request
from urllib.parse import urlparse
from urllib.parse import urljoin
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import tqdm

import pymorphy2

from polyglot.text import Text
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
class HTMLParser:
    def __init__(self, main_url=''):
        self.urls = []
        self.texts = []
        self.main_url = main_url
        if urlparse(main_url).scheme == '':
            self.main_url = 'https://' + self.main_url
        print('initialize HTMLParser with "{}"'.format(self.main_url))

    def _traverse(self, from_url=None):
        if from_url is None:
            from_url = self.main_url

        links = BeautifulSoup(urllib.request.urlopen(from_url)).find_all('a')
        for link in links:
            href = link.get('href')
            if href.startswith('#'):
                href = '/' + href
            url = urljoin(from_url, href)
            url = urlparse(url).geturl()
            is_external = url.startswith('http') and not url.startswith(self.main_url)
            if not is_external and not url in self.urls:
                yield url
                self._traverse(url)

    
    def _create_filename(self, url, i):
        url_str = url.replace('https://', '').replace('#', '').replace('.', '_').replace('/', '_').replace('-', '_')
        if i > 9999:
            print('WARNING: paragraph list is too long')
        return '{}_{:05d}'.format(url_str, i)
    
    
    def _save_text(self, text, path_to_save):
        with open(path_to_save, 'wb') as file:
            pickle.dump(text, file)
        #print('save texts to \"{}\"'.format(path_to_save))
        
    
    def parse(self, path_to_save='texts'):
        try:
            os.mkdir(path_to_save)
        except:
            print('folder "{}" already exists'.format(path_to_save))
        
        self.num_texts = 0
        for i, url in enumerate(self._traverse()):
            print('({}) {}'.format(i, url))
            try:
                soup =  BeautifulSoup(urllib.request.urlopen(url))
    
                # kill all script and style elements
                for script in soup(["script", "style"]):
                    script.extract()    # rip it out

                # get text
                text = soup.get_text()
                
                #replace eol, tabs, multiple whitespaces)
                text = re.sub('\\t', ' ', text)
                text = re.sub(' +', ' ', text)
                text = re.sub(' \n', '\n', text)

                #remove text in parentheses
                text = re.sub(r'\{[^}]*\}', '', text)
    
                #remove html comments
                #text = re.sub(re.compile("/\*.*?\*/",re.DOTALL ), "", text)
                #text = re.sub(re.compile("//.*?\n" ), "", text)
                
                #split to paragraphs of length >= 100
                paragraphs = filter(lambda x: len(x) >= 100, 
                                    map(lambda x: x.strip(), text.split('\n\n')))
                for i, p in enumerate(paragraphs):
                    p = p.strip()
                    #print('PARAGRAPH {}\n[ {} ]\n\n'.format(i + 1, p))
                    self._save_text(p, path_to_save=os.path.join(path_to_save, self._create_filename(url, i)))
                    self.num_texts += 1
            except:
                print('invalid url: {}'.format(url))
    
    def iterate_over_texts(self, load_from='texts'):
        print('iterate over {} saved texts'.format(self.num_texts))
        for filename in sorted(filter(lambda x: not x.startswith('.'), os.listdir(load_from))):
            with open(os.path.join(load_from, filename), 'rb') as file:
                yield (filename, pickle.load(file))

In [5]:
"""Base tokenizer/tokens classes and utilities."""

class Tokens(object):
    """A class to represent a list of tokenized text."""
    TEXT = 0
    TEXT_WS = 1
    SPAN = 2
    POS = 3
    LEMMA = 4
    NER = 5

    def __init__(self, data, annotators, opts=None):
        self.data = data
        self.annotators = annotators
        self.opts = opts or {}

    def __len__(self):
        """The number of tokens."""
        return len(self.data)

    def slice(self, i=None, j=None):
        """Return a view of the list of tokens from [i, j)."""
        new_tokens = copy.copy(self)
        new_tokens.data = self.data[i: j]
        return new_tokens

    def untokenize(self):
        """Returns the original text (with whitespace reinserted)."""
        return ''.join([t[self.TEXT_WS] for t in self.data]).strip()

    def words(self, uncased=False):
        """Returns a list of the text of each token
        Args:
            uncased: lower cases text
        """
        if uncased:
            return [t[self.TEXT].lower() for t in self.data]
        else:
            return [t[self.TEXT] for t in self.data]

    def offsets(self):
        """Returns a list of [start, end) character offsets of each token."""
        return [t[self.SPAN] for t in self.data]

    def pos(self):
        """Returns a list of part-of-speech tags of each token.
        Returns None if this annotation was not included.
        """
        if 'pos' not in self.annotators:
            return None
        return [t[self.POS] for t in self.data]

    def lemmas(self):
        """Returns a list of the lemmatized text of each token.
        Returns None if this annotation was not included.
        """
        if 'lemma' not in self.annotators:
            return None
        return [t[self.LEMMA] for t in self.data]

    def entities(self):
        """Returns a list of named-entity-recognition tags of each token.
        Returns None if this annotation was not included.
        """
        if 'ner' not in self.annotators:
            return None
        return [t[self.NER] for t in self.data]

    def ngrams(self, n=1, uncased=False, filter_fn=None, as_strings=True):
        """Returns a list of all ngrams from length 1 to n.
        Args:
            n: upper limit of ngram length
            uncased: lower cases text
            filter_fn: user function that takes in an ngram list and returns
              True or False to keep or not keep the ngram
            as_string: return the ngram as a string vs list
        """
        def _skip(gram):
            if not filter_fn:
                return False
            return filter_fn(gram)

        words = self.words(uncased)
        ngrams = [(s, e + 1)
                  for s in range(len(words))
                  for e in range(s, min(s + n, len(words)))
                  if not _skip(words[s:e + 1])]

        # Concatenate into strings
        if as_strings:
            ngrams = ['{}'.format(' '.join(words[s:e])) for (s, e) in ngrams]

        return ngrams

    def entity_groups(self):
        """Group consecutive entity tokens with the same NER tag."""
        entities = self.entities()
        if not entities:
            return None
        non_ent = self.opts.get('non_ent', '')
        groups = []
        idx = 0
        while idx < len(entities):
            ner_tag = entities[idx]
            # Check for entity tag
            if ner_tag != non_ent:
                # Chomp the sequence
                start = idx
                while (idx < len(entities) and entities[idx] == ner_tag):
                    idx += 1
                groups.append((self.slice(start, idx).untokenize(), ner_tag))
            else:
                idx += 1
        return groups

In [6]:
class Tokenizer(object):
    """Base tokenizer class.
    Tokenizers implement tokenize, which should return a Tokens class.
    """
    def tokenize(self, text):
        raise NotImplementedError

    def shutdown(self):
        pass

    def __del__(self):
        self.shutdown()


class RuTokenizer(Tokenizer):

    def __init__(self, **kwargs):
        """
        Args:
            annotators: set that can include pos, lemma, and ner.
        """
        self.annotators = copy.deepcopy(kwargs.get('annotators', set()))
        self.include_pos = {'pos'} & self.annotators
        self.include_lemma = {'lemma'} & self.annotators
        self.include_ner = {'ner'} & self.annotators
        self.morph = pymorphy2.MorphAnalyzer()
        self.wt = WhitespaceTokenizer()
        self.rt = RegexpTokenizer(r'\w+')

    def tokenize(self, text):
        #к нижнему регистру
        text = text.lower()
        
        # We don't treat new lines as tokens.
        clean_text = text.replace('\n', ' ')
        
        # remove punctuation
        clean_text = ' '.join(self.rt.tokenize(clean_text))
        
        # split by whitespaces and get spans
        spans = list(self.wt.span_tokenize(clean_text))
        n = len(spans)
        
        data = []
        for i in range(n):
            start_idx, end_idx = spans[i]
            
            token = clean_text[start_idx:end_idx]
        
            start_ws = start_idx  
            if i + 1 < n:
                end_ws = spans[i + 1][0]
            else:
                end_ws = start_idx + len(token)
            
            token_ws = clean_text[start_ws:end_ws]
            
            lemma, pos, ent_type = '', '', ''
            if self.include_pos or self.include_lemma:
                p = self.morph.parse(token)[0]
                if self.include_lemma:
                    lemma = p.normal_form
                if self.include_pos:
                    pos = p.tag.POS
            
            
            if self.include_ner:
                entities = Text(token, hint_language_code='ru').entities
                if len(entities):
                    ent_type = entities[0].tag
            
            data.append((token, token_ws, spans[i], pos, lemma, ent_type))

        return Tokens(data, self.annotators, opts={'non_ent': ''})

In [7]:
tokenizer = RuTokenizer(annotators={'lemma', 'pos', 'ner'})

In [8]:
class QA:
    
    def __init__(self, url):
        self.parser = HTMLParser(url)
    
    
    def parse(self):
        self.parser.parse()
        

    def calculate_overlap(self, p_tokens, q_tokens):
        p_set = set(p_tokens.words())
        q_set = set(q_tokens.words())
        return len(p_set.intersection(q_set))

    
    def find_closest_paragraph(self, question):
        question_tokens = tokenizer.tokenize(question)
        max_overlap = None
        max_match_paragraph = None
        for filename, text in self.parser.iterate_over_texts():
            paragraph_tokens = tokenizer.tokenize(text)
            overlap = self.calculate_overlap(paragraph_tokens, question_tokens)
            if max_overlap is None or overlap > max_overlap:
                max_overlap = overlap
                max_match_paragraph = text

        return max_match_paragraph
    
    
    def answer(self, question):
        return self.find_closest_paragraph(question)

In [15]:
class TfIdfQA:
    def __init__(self, url):
        self.parser = HTMLParser(url)
        self.parser.parse()
        self.texts = []
        for filename, text in self.parser.iterate_over_texts():
            self.texts.append(text.lower())
            
        self.tfIdf = TfidfVectorizer()
        self.tokens = self.tfIdf.fit_transform(self.texts)
        self.word_dict = self.tfIdf.vocabulary_
        
    def _build_one_hot_vector_by_string_(self, string):
        description = np.zeros(len(self.word_dict))
        for word in string.split():
            if word in self.word_dict:
                description[self.word_dict[word]] = 1
        return description

    def get_best_indx(self, question):
        oh = self._build_one_hot_vector_by_string_(question)
        scores = (self.tokens * oh)
        #print((self.tokens * oh).shape)
        #print(scores.shape)
        
        return np.argmax(scores)
        #return len(p_set.intersection(q_set))

    
    def find_closest_paragraph(self, question):
        return self.texts[self.get_best_indx(question)]
    
    def answer(self, question):
        return self.find_closest_paragraph(question)

    

In [16]:
URL = "https://tutorial.djangogirls.org/ru/"

In [17]:
qa = QA(url=URL)

initialize HTMLParser with "https://tutorial.djangogirls.org/ru/"


In [18]:
qa.parse()

folder "texts" already exists


/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /usr/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


(0) https://tutorial.djangogirls.org/ru/
(1) https://tutorial.djangogirls.org/ru/installation/
(2) https://tutorial.djangogirls.org/ru/how_the_internet_works/
(3) https://tutorial.djangogirls.org/ru/intro_to_command_line/
(4) https://tutorial.djangogirls.org/ru/python_installation/
(5) https://tutorial.djangogirls.org/ru/code_editor/
(6) https://tutorial.djangogirls.org/ru/python_introduction/
(7) https://tutorial.djangogirls.org/ru/django/
(8) https://tutorial.djangogirls.org/ru/django_installation/
(9) https://tutorial.djangogirls.org/ru/django_start_project/
(10) https://tutorial.djangogirls.org/ru/django_models/
(11) https://tutorial.djangogirls.org/ru/django_admin/
(12) https://tutorial.djangogirls.org/ru/deploy/
(13) https://tutorial.djangogirls.org/ru/django_urls/
(14) https://tutorial.djangogirls.org/ru/django_views/
(15) https://tutorial.djangogirls.org/ru/html/
(16) https://tutorial.djangogirls.org/ru/django_orm/
(17) https://tutorial.djangogirls.org/ru/dynamic_data_in_templa

In [19]:
tfIdfQa = TfIdfQA(url=URL)

initialize HTMLParser with "https://tutorial.djangogirls.org/ru/"
folder "texts" already exists


/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /usr/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


(0) https://tutorial.djangogirls.org/ru/
(1) https://tutorial.djangogirls.org/ru/installation/
(2) https://tutorial.djangogirls.org/ru/how_the_internet_works/
(3) https://tutorial.djangogirls.org/ru/intro_to_command_line/
(4) https://tutorial.djangogirls.org/ru/python_installation/
(5) https://tutorial.djangogirls.org/ru/code_editor/
(6) https://tutorial.djangogirls.org/ru/python_introduction/
(7) https://tutorial.djangogirls.org/ru/django/
(8) https://tutorial.djangogirls.org/ru/django_installation/
(9) https://tutorial.djangogirls.org/ru/django_start_project/
(10) https://tutorial.djangogirls.org/ru/django_models/
(11) https://tutorial.djangogirls.org/ru/django_admin/
(12) https://tutorial.djangogirls.org/ru/deploy/
(13) https://tutorial.djangogirls.org/ru/django_urls/
(14) https://tutorial.djangogirls.org/ru/django_views/
(15) https://tutorial.djangogirls.org/ru/html/
(16) https://tutorial.djangogirls.org/ru/django_orm/
(17) https://tutorial.djangogirls.org/ru/dynamic_data_in_templa

In [20]:
question = 'Как добавить определения блоков для селекторов?'

In [24]:
qa.answer(question)

iterate over 447 saved texts


'Теперь добавим определения блоков для различных селекторов. Селекторы, которые начинают с символа ., относятся к классам. В Интернете много хороших справочников по CSS, которые могут помочь тебе понять следующий код. А сейчас просто скопируй и вставь код в файл djangogirls/static/css/blog.css:\nblog/static/css/blog.css\n.page-header'

In [25]:
tfIdfQa.answer(question)

'мы хотим использовать это фрагмент в твоём шаблоне для отображения содержимого. пора добавить теги блоков в этот файл!\nнам нужно, чтобы новый тег блока соответствовал тегу в файле base.html. также нам необходимо включить весь код, который соответствует твоему блоку с содержимым. для этого расположи всё между  и . вот так:\nblog/templates/blog/post_list.html'